In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.metrics.pairwise import cosine_similarity

Using TensorFlow backend.


In [3]:
df = pd.read_csv('1.csv')
df = df[['name','rate','location','dish_liked','cuisines','approx_cost','neighbourhood']]
df.drop_duplicates('name', inplace=True)
df.dropna(subset=['cuisines'], inplace=True)

In [4]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["cuisines"] = df["cuisines"].apply(lambda text: remove_punctuation(text))
df['addr'] = df['neighbourhood'] +' '+ df['location']
df.set_index('name', inplace=True)
indices = pd.Series(df.index)

In [5]:
df

,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Jalsa,4.1,Banashankari,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",North Indian Mughlai Chinese,800.0,Banashankari,Banashankari Banashankari
Spice Elephant,4.1,Banashankari,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",Chinese North Indian Thai,800.0,Banashankari,Banashankari Banashankari
San Churro Cafe,3.8,Banashankari,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",Cafe Mexican Italian,800.0,Banashankari,Banashankari Banashankari
Addhuri Udupi Bhojana,3.7,Banashankari,Masala Dosa,South Indian North Indian,300.0,Banashankari,Banashankari Banashankari
Grand Village,3.8,Basavanagudi,"Panipuri, Gol Gappe",North Indian Rajasthani,600.0,Banashankari,Banashankari Basavanagudi
...,...,...,...,...,...,...,...
Chime - Sheraton Grand Bengaluru Whitefield Hotel &...,4.3,"ITPL Main Road, Whitefield","Cocktails, Pizza, Buttermilk",Finger Food,2500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
The Nest - The Den Bengaluru,3.4,"ITPL Main Road, Whitefield",NaN,Finger Food North Indian Continental,1500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
Nawabs Empire,3.2,"ITPL Main Road, Whitefield",NaN,North Indian Chinese Arabian Momos,300.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"


In [6]:
l = [i for i in df['addr']]
l2 = [i for i in df['cuisines']]
l3 = [i for i in df['dish_liked']]

In [7]:
dish=[]
for i in l3:
    if pd.isna(i):
        dish.append('')
    else:
        dish.append(''.join(i.split(',')))

In [8]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in l]
encoded_docs2 = [one_hot(d, vocab_size) for d in l2]
encoded_docs3 = [one_hot(d, vocab_size) for d in dish]

max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs2 = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
padded_docs3 = pad_sequences(encoded_docs3, maxlen=max_length, padding='post')


In [20]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

ERROR! Session/line number was not unique in database. History logging moved to new session 66
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 8)             400       
_________________________________________________________________
flatten_2 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 81        
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# fit the model
embd = model.predict(padded_docs)
embd2 = model.predict(padded_docs2)
embd3 = model.predict(padded_docs3)

In [22]:
embd,embd2,embd3

(array([[0.49893546],
        [0.49893546],
        [0.49893546],
        ...,
        [0.50681305],
        [0.5072301 ],
        [0.50128514]], dtype=float32),
 array([[0.5115451 ],
        [0.51999784],
        [0.5160187 ],
        ...,
        [0.5146013 ],
        [0.5086273 ],
        [0.49686727]], dtype=float32),
 array([[0.49787268],
        [0.5062703 ],
        [0.49920198],
        ...,
        [0.50141346],
        [0.50141346],
        [0.50141346]], dtype=float32))

In [27]:
sim,sim2,sim3 = [],[],[]
for i in range(indices.shape[0]):
    #sim.append([embd[i][0],embd2[i][0],embd3[i][0]])
    sim.append([embd[i][0],embd2[i][0],embd3[i][0]]) 

In [28]:
sim[0]

[0.49893546, 0.5115451, 0.49787268]

In [31]:
def rec(name):
    idx = indices[indices == name].index[0]
    corr=[]
    for i in range(len(sim)):
        #corr.append( (cosine_similarity([sim[idx]],[sim[i]])[0][0], i))
        corr.append( (cosine_similarity([sim[idx]],[sim[i]]), i))
     
    corr = sorted(corr,key = lambda x:x[0], reverse=True)
    corr = [c[1] for c in corr]
    corr.remove(idx)
    
    return df.iloc[corr[:11]]
    

    
rec('Jalsa')

ERROR! Session/line number was not unique in database. History logging moved to new session 67


,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Refills Eatery And Beverages,3.8,BTM,NaN,North Indian South Indian Chinese,450.0,BTM,BTM BTM
Crunchy Nights,3.6,BTM,NaN,North Indian Chinese Continental,350.0,BTM,BTM BTM
Coffee Time,NaN,BTM,NaN,Sandwich Juices Burger,150.0,BTM,BTM BTM
Rayalaseema Abhiruchulu,NaN,BTM,NaN,North Indian South Indian Chinese,250.0,BTM,BTM BTM
Jaishree Pavilion,3.7,Thippasandra,NaN,North Indian South Indian Chinese,700.0,Indiranagar,Indiranagar Thippasandra
Aburrahaman,NaN,Thippasandra,NaN,North Indian South Indian Chinese,400.0,Indiranagar,Indiranagar Thippasandra
BISO - Citrus Hotels,3.0,Bellandur,"Salads, Gulab Jamun, Pasta, Fish, Rasgulla, Co...",North Indian Italian Chinese Asian,1000.0,Bellandur,Bellandur Bellandur
Namakh Store And Cafe,3.7,New BEL Road,NaN,Cafe Continental American,400.0,Malleshwaram,Malleshwaram New BEL Road
David's Kitchen,3.5,BTM,NaN,North Indian South Indian Chinese,200.0,Basavanagudi,Basavanagudi BTM


In [170]:
cosine_similarity([[11,1]],[[52,2]])[0][0]

0.9986369866844395